In [1]:
import os
import pandas as pd

from tfcomb import CombObj
import tfcomb.objects
import tfcomb.annotation

pd.set_option("display.max_columns", 50)

In [24]:
# list treatment folder
directory = '/Users/user/Desktop/tomato_atac/tfcomb2/bed/similar'
gff_path = '/Users/user/Desktop/tomato_atac'
directory_contents = [name for name in os.listdir(directory) if os.path.isdir(os.path.join(directory, name))]
print(directory_contents)

['similar_ATAC_M82_6h_vs_ATAC_M82_0h_proximal_up', 'similar_ATAC_M82_1h_vs_ATAC_M82_0h_proximal_down', 'similar_ATAC_M82_1h_vs_ATAC_M82_0h_distal_down', 'similar_ATAC_M82_6h_vs_ATAC_M82_0h_proximal_down', 'similar_ATAC_M82_1h_vs_ATAC_M82_0h_proximal_up', 'similar_ATAC_M82_6h_vs_ATAC_M82_0h_distal_down', 'similar_ATAC_M82_6h_vs_ATAC_M82_0h_distal_up', 'similar_ATAC_M82_1h_vs_ATAC_M82_0h_distal_up']


In [25]:
# annotate binding sites to genes
locations = pd.DataFrame()

# custom: annotation genes
custom_config = {"queries": [{"distance": [1500, 500], 
    "feature_anchor": "start", 
    "feature": "transcript"}],
    "priority": True, 
    "show_attributes": "all"}

# the transcript file was made by following code:
# gffread Slycopersicum_796_ITAG5.0.gene.gff3 -T -o Slycopersicum_796_ITAG5.0.gene.gtf
# awk '$3 == "transcript"' Slycopersicum_796_ITAG5.0.gene.gtf > Slycopersicum_796_ITAG5.0.sorted_transcript.gtf


for dir in directory_contents:
    file = os.path.join(directory, dir, 'selected_sig', dir + '.pkl')
    print(file)
    # extract base name
    bn = os.path.splitext(file)[0]

    ###### LAOD FILES FROM PKL ######
    C = tfcomb.CombObj().from_pickle(os.path.join(file))
    
    # roughly show the data
    C.rules.head(20)

    # pass all the rules
    table = C.rules
    ############

    ###### GET LOCATION TABLE ######
    # get location table
    for index, row in table.iterrows():
        tf1 = row['TF1']
        tf2 = row['TF2']
        print(f'Processing: TF1={tf1}, TF2={tf2}')

        location = C.get_pair_locations((tf1, tf2)).as_table()
        locations = pd.concat([locations, location], axis = 0)

    locations.reset_index(drop = True, inplace = True)
    print(locations)
    if not locations.empty:
        print("location table has rows and keep going.")
    else:
        print("location table is empty. stop!")
        # Skip to the next iteration if the DataFrame is not empty
        continue
    ############
    
    
    ###### ANNOTATION ######
    # annotate genes
    annotated = tfcomb.annotation.annotate_regions(locations, gtf = os.path.join(gff_path, 'Slycopersicum_796_ITAG5.0.sorted_transcript.gtf'), config = custom_config)
    
    if annotated is None:
        print("annotated table is empty. stop!")
        # Skip to the next iteration if the DataFrame is not empty
        continue
    
    # export table
    print("file is exported to " + os.path.join(directory, bn + '_sites.txt'))
    annotated.to_csv(os.path.join(directory, bn + '_sites.txt'),
                sep='\t', index=False, header=True)
    ############
    print('Finished: ' + bn + '!')
print('Finished all!')

/Users/user/Desktop/tomato_atac/tfcomb2/bed/similar/similar_ATAC_M82_6h_vs_ATAC_M82_0h_proximal_up/selected_sig/similar_ATAC_M82_6h_vs_ATAC_M82_0h_proximal_up.pkl
Empty DataFrame
Columns: []
Index: []
location table is empty. stop!
/Users/user/Desktop/tomato_atac/tfcomb2/bed/similar/similar_ATAC_M82_1h_vs_ATAC_M82_0h_proximal_down/selected_sig/similar_ATAC_M82_1h_vs_ATAC_M82_0h_proximal_down.pkl
Processing: TF1=C2_MYBrelated_2, TF2=C3_AP2EREBP_66
INFO: Setting up binding sites for counting


Processing: TF1=C3_AP2EREBP_15, TF2=C3_AP2EREBP_23
Processing: TF1=C3_AP2EREBP_23, TF2=C3_Trihelix_25
Processing: TF1=C3_AP2EREBP_23, TF2=C3_Trihelix_26
   site1_chrom  site1_start  site1_end       site1_name  site1_score   
0            1     89978391   89978400  C2_MYBrelated_2     10.80361  \
1            7       654144     654152   C3_AP2EREBP_23      6.70273   
2            7       654144     654152   C3_AP2EREBP_23      6.70273   
3            7       654144     654152   C3_AP2EREBP_23      6.70273   

  site1_strand  site2_chrom  site2_start  site2_end      site2_name   
0            +            1     89978404   89978411  C3_AP2EREBP_66  \
1            +            7       654167     654174  C3_AP2EREBP_15   
2            +            7       654167     654174  C3_Trihelix_25   
3            +            7       654167     654175  C3_Trihelix_26   

   site2_score site2_strand  site_distance site_orientation  
0     10.75965            -              4       convergent  
1     